In [ ]:
# import the modules
import os
from nilearn import datasets 
from nilearn import plotting as nplot
from nilearn import image as nimg
from nilearn.maskers import NiftiLabelsMasker
from nilearn.connectome import ConnectivityMeasure
from nilearn.interfaces.fmriprep import load_confounds
import nibabel as nib
import numpy as np
import pandas as pd
import bids
import matplotlib.pyplot as plt

In [ ]:
# change to working dir
data_dir ='/Users/wang/Desktop/Research_projects/BBSC/Functional/Parcellation/data/cleaned_data'
os.chdir(data_dir)

In [ ]:
# extract the data form the atlas, sub3 was excluded because head movement
indi_parc = '/Users/wang/Desktop/Research_projects/BBSC/Functional/Parcellation/1_generate_profiles_and_ini_params/group/sub01indi_network.nii.gz'

import glob
sub_list = ['sub-01']
masker = NiftiLabelsMasker(labels_img=indi_parc,verbose=3)
all_data = []

for subi,sub_name in enumerate(sub_list):
    data_path = glob.glob('./'+ sub_name + '/*.nii.gz')
    data_path = sorted(data_path)
    all_ses_data = []
    
    for datai, data_name in enumerate(data_path):
        time_series = masker.fit_transform (data_name)
        all_ses_data.append(time_series)

    all_data.append (all_ses_data)    
    
print(len(all_ses_data))    
print(len(all_data))   

In [ ]:
#--------- extract the data, here compute the data from subject 2
sub_data = all_data[0]
sub_all_data = np.array(sub_data)

session2delete = [10,13] # session 11 and 14 were deleted because sleep for sub1
# session2delete = [9,11,12,13,14,19,21,22,27,29,34,37] # sessions 10,12-15,20,22,23,28,30,35,38 were deleted for sub2
sub_all_data = np.delete(sub_all_data, session2delete, axis=0) # session * fmri_data * networks

# sub_all_data [session2delet,:,:] = np.nan
sub_all_data_avg = np.nanmean(sub_all_data, axis=0)
# sub_all_data [session2delet,:,:] = 0

In [ ]:
sub_all_data.shape

In [ ]:
# plot the first network time course in the first session
import seaborn as sns
data2plot = pd.DataFrame(
                        {"Session1":np.squeeze(sub_all_data[0,:,0]),
                        "Session_avg":sub_all_data_avg[:,0]}
                        )
sns.lineplot(data=data2plot[['Session1','Session_avg']])

In [ ]:
# compute the inter session variablity for different networks
correlation_measure = ConnectivityMeasure(kind='correlation')

labels=np.arange(1, 38+1).tolist() # for sub1
# labels=np.arange(1, 40+1).tolist()   # for sub2

labels=np.delete(labels, session2delete) 

net_corr = []



for neti in range(sub_all_data.shape[2]):
    
    net_time_series = np.transpose(np.squeeze(sub_all_data[:,:,neti]))

    #correlation_matrix = np.corrcoef(net_time_series, rowvar=False)
    correlation_matrix = correlation_measure.fit_transform([net_time_series])[0]
    np.fill_diagonal(correlation_matrix, 0)
    nplot.plot_matrix(correlation_matrix, figure=(5, 5), labels=labels,
                     vmax=0.2, vmin=-0.2, title=None,
                     reorder=False)
    plt.savefig('/Users/wang/Desktop/UiB/Manuscripts/BBSC/Long_night/figures/' + f'Network_{neti+1}.tiff', dpi=300)
    plt.close()

In [ ]:
# compute the correlaiton between each session and session average
ses_corr = []
for sesi in range(sub_all_data.shape[0]):
    df1 = pd.DataFrame(np.squeeze(sub_all_data[sesi,:,:]))
    df2 = pd.DataFrame(sub_all_data_avg)
    df_corr = df1.corrwith(df2)
    ses_corr.append(df_corr)
ses_corr = np.array(ses_corr)

In [ ]:
# plot the correlation between sessions and average for each network
import seaborn as sns
corr_map=pd.DataFrame(ses_corr,columns=['Visual', 'Somatomotor', 'Dorsal Attention',
                                        'Salience / Ventral Attention',
                                        'Limbic','Control','Default'])

network_colors = {
    'Visual': (120, 18, 134),
    'Somatomotor': (70, 130, 180),
    'Dorsal Attention': (0, 118, 14),
    'Salience': (196, 58, 250),
    'Limbic': (220, 248, 164),
    'Control': (230, 148, 34),
    'Default': (205, 62, 78),
}

# Assuming corr_map is your data for the violin plot

# Convert RGB values to normalized [0, 1] range
normalized_colors = [(r / 255, g / 255, b / 255) for r, g, b in network_colors.values()]

#plt.figure(figsize=(3, 3))
network_plots = sns.violinplot(corr_map, palette=normalized_colors)
network_plots.set_xticklabels(network_plots.get_xticklabels(), rotation=45, ha='right')
plt.yticks([-0.2, 0, 0.2, 0.4, 0.6])
plt.ylim(-0.25, 0.65)

plt.savefig('/Users/wang/Desktop/UiB/Manuscripts/BBSC/Long_night/figures/Network_corr.tiff', dpi=300)
plt.show()